$$e_{ij} =$$

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchtext
from torchtext.data import Field,Iterator

In [2]:
def tokenize(text):
    return text.split(' ')

In [3]:
TEXT = Field(tokenize=tokenize,use_vocab=True,batch_first=True)
TAG = Field(tokenize=tokenize,use_vocab=True,unk_token=None,batch_first=True)

In [4]:
train_data = torchtext.data.TabularDataset(path='data/ner_train.txt',
                                                               format='tsv', # \t로 구분
                                                               fields=[('input',TEXT),('target',TAG)])

In [5]:
one = train_data.examples[0]
print(one.input)
print(one.target)

['약', '4', '일', '간', '놀', '러', '갈', '맛집', '여행']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [6]:
TEXT.build_vocab(train_data)
TAG.build_vocab(train_data)

In [7]:
len(TEXT.vocab)

324

In [8]:
TAG.vocab.itos

['<pad>',
 'O',
 'B-region',
 'I-region',
 'B-food',
 'I-price',
 'B-price',
 'I-food']

In [9]:
def index2word(index):
    return [TEXT.vocab.itos[i] for i in index]

In [10]:
BATCH_SIZE = 32
BPTT_LEN = 10

train_loader = Iterator(train_data,batch_size=BATCH_SIZE,sort_key=lambda x: len(x.input),sort_within_batch=True,repeat=False,device=-1)

In [11]:
class RNNLM(nn.Module):
    def __init__(self,input_size,embed_size,hidden_size,output_size,num_layers=1,bidirec=False,dropout=0.5):
        super(RNNLM,self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        if bidirec:
            self.num_directions = 2
        else:
            self.num_directions = 1
            
        self.embed = nn.Embedding(input_size,embed_size)
        self.gru = nn.GRU(embed_size,hidden_size,num_layers,batch_first=True,bidirectional=bidirec)
        self.linear = nn.Linear(hidden_size*self.num_directions,output_size)
        self.dropout = nn.Dropout(dropout)
        
    def init_hidden(self,batch_size):
        # (num_layers * num_directions, batch_size, hidden_size)
        hidden = Variable(torch.zeros(self.num_layers*self.num_directions,batch_size,self.hidden_size))
        return hidden
    
    def forward(self,inputs):
        """
        inputs : B,T
        """
        embed = self.embed(inputs) # word vector indexing
        embed = self.dropout(embed)
        hidden = self.init_hidden(inputs.size(0))
        output, hidden = self.gru(embed,hidden)
        
        # Many-to-Many
        output = self.dropout(output)
        output = self.linear(output) # B,T,H -> B,T,V
        output = output.contiguous().view(output.size(0)*output.size(1),-1) # B*T,V
        return output

In [12]:
STEP = 100
VOCAB = len(TEXT.vocab)
TAGSET = len(TAG.vocab)
EMBED = 50
HIDDEN = 50
LR = 0.001

model = RNNLM(VOCAB,EMBED,HIDDEN,TAGSET,bidirec=True)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=LR)

In [13]:
model.train()
for step in range(STEP):
    losses=[]
    for i, batch in enumerate(train_loader):
        inputs = batch.input
        targets = batch.target
        model.zero_grad()
        preds = model(inputs)
        loss = loss_function(preds,targets.contiguous().view(-1))
        losses.append(loss.data[0])
        loss.backward()
        optimizer.step()
    
    if step % 10==0:
        print("mean loss : %.3f" % np.mean(losses))

mean loss : 2.094
mean loss : 0.557
mean loss : 0.346
mean loss : 0.312
mean loss : 0.267
mean loss : 0.232
mean loss : 0.197
mean loss : 0.195
mean loss : 0.162
mean loss : 0.142


## Test 

In [14]:
from konlpy.tag import Kkma
tagger = Kkma()

In [15]:
test = "강남역 근처 저렴한 맛집 좀 추천해줘"
tokenized = tagger.morphs(test)

In [16]:
input = TEXT.numericalize([tokenized],train=False,device=-1)
input

Variable containing:

Columns 0 to 12 
    0    94    25    13   108     0    10     0     9     3   108     0     4

Columns 13 to 13 
    0
[torch.LongTensor of size 1x14]

In [17]:
pred = model(input)
pred = pred.max(1)[1]

In [18]:
[TAG.vocab.itos[i] for i in pred.data.tolist()]

['B-region',
 'B-region',
 'O',
 'B-price',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']